In [1]:
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import pandas as pd


class Server:
    def __init__(self, server_url="https://ppc.tecsci.com.br/api/v1.0"):
        self.server_url = server_url
        self.token = None
        self.response = None
        self.excecao = None
        self.login()



    def login(self):
        try:
            response = requests.post(
                self.server_url + "/auth/login",
                headers={'Content-Type': 'application/json'}, 
                json={'username': 'gustavo.elias', 'password': '12345678'}
            )

            # Armazena status e texto da resposta para debugging
            self.response = response.text

            if response.status_code != 200:
                self.excecao = f"Erro {response.status_code} - {response.reason}"
                return
            
            self.token = response.json().get("access_token")

        except Exception as e:
            self.excecao = e.args
    
    def _get_request(self, endpoint, params=None):
        return requests.get(self.server_url + endpoint, 
                            headers= {"Authorization": f"Bearer {self.token}"},
                            params = params
                            ).json()
    

    def _post_request(self, endpoint, data=None):
        return requests.post(self.server_url + endpoint, 
                            json = data, 
                            headers= {"Authorization": f"Bearer {self.token}", "Content-Type": "application/json"})


    def get_burnin_data(self, data):
        if not self.token:
            return
        db_data = []
        page = 1
        last_datetime = data
        while True:
            response = requests.get(self.server_url + "/teste_burnin", 
                                    headers={'Content-Type': 'application/json',
                                            'Authorization': f'Bearer {self.token}', 
                                            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"},
                                    params={"per_page": 100,
                                            "page": page,
                                            "filter": f"datetime >= '{last_datetime}'"
                                            }).json()
            if not response:
                break
            db_data.extend(response)
            page += 1
        df = pd.json_normalize(db_data)
        if df.empty:
            return df
        df['horario'] = pd.to_datetime(df['horario'], format='ISO8601')
        df = df.drop_duplicates(subset=['controladora_id'], keep="last")\
            .reset_index()
        df = df[~df['operador_id.nome'].isin(['Charles', 'Valciscley'])]
        return df
    
    def get_communication_data(self, data):
        db_data = []
        page = 1
        last_datetime = data
        while True:
            response = requests.get(self.server_url + "/teste_firmware", 
                                    headers={'Content-Type': 'application/json',
                                            'Authorization': f'Bearer {self.token}', 
                                            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"},
                                    params={"per_page": 100,
                                            "page": page,
                                            "filter": f"datetime >= '{last_datetime}'"
                                            }).json()
            if not response:
                break
            db_data.extend(response)
            page += 1
        df = pd.json_normalize(db_data)
        df['horario'] = pd.to_datetime(df['horario'], format='ISO8601')
        df = df.drop_duplicates(subset=['controladora_id'], keep="last")\
            .reset_index()
        df = df[~df['operador_id.nome'].isin(['Charles', 'Valciscley'])]
        return df
    
    def get_power_data(self, data):
        db_data = []
        page = 1
        last_datetime = data
        while True:
            response = requests.get(self.server_url + "/teste_potencia", 
                                    headers={'Content-Type': 'application/json',
                                            'Authorization': f'Bearer {self.token}', 
                                            'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"},
                                    params={"per_page": 100,
                                            "page": page,
                                            "filter": f"datetime >= '{last_datetime}'"
                                            }).json()
            if not response:
                break
            db_data.extend(response)
            page += 1
        df = pd.json_normalize(db_data)
        df['horario'] = pd.to_datetime(df['horario'], format='ISO8601')
        df = df.drop_duplicates(subset=['controladora_id'], keep="last")\
            .reset_index()
        df = df[~df['operador_id.nome'].isin(['Charles', 'Valciscley'])]
        return df
    

    
    def get_status_data(self):
        db_data = []
        page = 1
        while True:
            response = requests.get(self.server_url + "/tcu", 
                                    headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {self.token}'},
                                    params={"per_page": 100, "page": page}).json()
            if not response:
                break
            db_data.extend(response)
            page += 1
        df = pd.json_normalize(db_data)
        df = df.dropna(subset=['status.status_id.nome'])
        return df
    
    def get_tcu_history(self, serial_number):
        response = requests.get(self.server_url + f"/tcu/{serial_number}", 
                                headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {self.token}'},
                                ).json().get("status")
        df = pd.json_normalize(response)
        df['updated_at'] = pd.to_datetime(df['updated_at'])
        df = df.sort_values('updated_at')
        df = df[["status_id.nome", "operador_id.nome", "created_at"]]
        
        return df

    def get_operators(self):
        operadores = [{"nome": user["nome"].rstrip(), "id": user["id"]} 
                      for user in requests.get(self.server_url + "/operador", headers={'Authorization': f'Bearer {self.token}'}).json()]
        return operadores
    
    def get_locals(self):
        locals = [{"nome": local["nome"].rstrip(), "id": local["id"]} 
                      for local in requests.get(self.server_url + "/status", headers={'Authorization': f'Bearer {self.token}'}).json()]
        return locals
    

    def change_tcu_status(self, serial_number, operator_id, status_id):
        data = {
            "controladora_id": serial_number,
            "status_id": status_id,
            "operador_id": operator_id
        }
        response = requests.post(self.server_url + "/status_tcu", json=data,  headers={'Authorization': f'Bearer {self.token}'})
        if response.status_code == 201:
            return {"success": True, "message": "Status alterado com sucesso!"}
        else:
            # Tentar retornar uma mensagem mais detalhada do servidor (se disponível)
            error_message = response.json().get("message", "Erro desconhecido")
            return {"success": False, "message": f"Falha ao alterar status: {error_message}"}
        

    def post_operator(self, nome, username):
        data = {
            "nome": nome,
            "empresa_id": 1,
            "username": username,
            "password": "12345678"
        }
        return self._post_request("operador", data)
    
    
    def get_etapa(self):
        return self._get_request("etapa")
        
    def post_etapa(self, nome):
        data = {
            "nome": nome
        }
        return self._post_request("etapa", data)
    
    def get_erro(self):
        return self._get_request("erro")
    
    def post_erro(self, etapa_id, nome):
        data = {
            "etapa_id": etapa_id,
            "nome": nome
        }
        return self._post_request("erro", data)
    

    def get_rft(self):
        return self._get_request("rft")
    
    def post_rft(self, controladora_id, operador_id, error_id, horario):
        data = {
            "controladora_id": controladora_id,
            "operador_id": operador_id,
            "erro_id": error_id,
            "horario": horario
        }
        return self._post_request("rft", data)


    def get_materials(self):
        return self._get_request("material")
    
    def post_material(self, codigo, descricao):
        data = {
            "id": codigo,
            "nome": descricao
        }
        return self._post_request("material", data)
    
    def get_solucao(self):
        return self._get_request("solucao")
    
    def post_solucao(self, nome):
        data = {
            "nome": nome
        }
        return self._post_request("solucao", data)
    

    def get_manutencao(self):
        return self._get_request("manutencao")

    def post_manutencao(self, operador_id, rft_id, solucao_id, horario, duracao):
        data = {
            "operador_id": operador_id,
            "rft_id": rft_id,
            "solucao_id": solucao_id,
            "horario": horario,
            "duracao": duracao
        }
        return self._post_request("manutencao", data)

    def get_perdas(self):
        return self._get_request("perdas")
    
    def post_perdas(self, material_id, manutencao_id, quantidade):
        data = {
            "material_id": material_id,
            "manutencao_id": manutencao_id,
            "quantidade": quantidade
        }
        return self._post_request("perdas", data)


In [2]:
server = Server("http://127.0.0.1:8087/api/v1.0/")


In [7]:
# ETAPAS
etapas = [
    "Pré-testes",
    "Teste de Potência",
    "Teste de Comunicação",
    "Burn In"
]

for etapa in etapas:
    server.post_etapa(etapa)


In [3]:
operadores = [
    # ("Joao", "joao.joao"),
    # ("Thiago", "thiago.thiago"),
    # ("Charles", "charles.charles"),
    # ("Valciscley", "val.val"),
    ("Lucas Villani", "lucas.villani"),
    ("Mario", "mario.mario"),
    ("Fabiana", "fabiana.fabiana"),
    ("Kathleyn", "kathleyn.kathleyn"),
    ("Vinicius R", "vinicius.vinicius"),
    ("Silas", "silas.silas"),
    ("Thiago Dias", "thiago.dias"),
]

for nome, username in operadores:
    server.post_operator(nome, username)

In [ ]:
#ERROS
erros = [
    (1, "TCU com corrente baixa"),
    (2, "Erro no DCDC"),
    (3, "LED inoperante"),
    (3, "Erro na Ponte H"),
    (3, "TCU não comunica"),
    (3, "Erro no ADC"),
    (4, "LED invertido"),
    (4, "TCU não movimentou"),
    (4, "TCU não ligou na bateria"),
    (4, "Comunicação serial intermitente"),
]

for etapa_id, nome in erros:
    server.post_erro(etapa_id, nome)

In [14]:
rfts = [
    ("0225020108196", 7, 2, "2025-03-24 08:00:00"),
    ("0225020108103", 8, 1, "2025-03-25 08:00:00"),
    ("0225020112193", 7, 3, "2025-03-24 08:00:00"),
]

for controlada_id, operador_id, error_id, datetime in rfts:
    response = server.post_rft(controladora_id=controlada_id, operador_id=operador_id, error_id=error_id, horario=datetime)


In [31]:
materiais = [
    ("R104", "Resistor 100k"),
    ("U16", "Lore ipslum ")
]

for codigo, descricao in materiais:
    server.post_material(codigo=codigo, descricao=descricao)

In [38]:
solucoes = ["Ressolda DCDC", "Ressolda C6", "Substituir U14 e R104"]

for solucao in solucoes:
    server.post_solucao(nome=solucao)

In [8]:
manutencoes = [
    (9, 4, 1, "2025-03-24 08:00:00", 40.0),
    (9, 5, 2, "2025-03-24 08:00:00", 50.0),
    (10, 6, 3, "2025-03-24 08:00:00", 60.0),
]

for  operador_id, rft_id, solucao_id, horario, duracao in manutencoes:
    server.post_manutencao(operador_id=operador_id,
                            rft_id=rft_id, solucao_id=solucao_id, horario=horario, duracao=duracao)

In [ ]:
perdas = [
    ("R104", 9, 1),
    ("U16", 9, 2),
]


for material_id, manutencao_id, quantidade in perdas:
    server.post_perdas(material_id=material_id, manutencao_id=manutencao_id, quantidade=quantidade)

In [ ]:
import json

response = server.get_perdas()
print(json.dumps(response, indent=4))

[{'id': 3, 'manutencao_id': {'duracao': 60.0, 'horario': '2025-03-24T08:00:00+00:00', 'id': 9, 'operador_id': {'empresa': {'id': 1, 'nome': 'TECSCI'}, 'id': 10, 'nome': 'Valciscley'}, 'rft': {'controladora_id': '0225020112193', 'erro_id': {'etapa': {'id': 2, 'nome': 'Teste de Comunicação'}, 'id': 3, 'nome': 'Falha no ADC'}, 'horario': '2025-03-24T08:00:00+00:00', 'id': 6, 'operador_id': {'empresa': {'id': 1, 'nome': 'TECSCI'}, 'id': 7, 'nome': 'Joao'}}, 'solucao': {'id': 3, 'nome': 'Substituir U14 e R104'}}, 'material_id': {'id': 'U16', 'nome': 'Lore ipslum'}, 'quantidade': 2}, {'id': 5, 'manutencao_id': {'duracao': 60.0, 'horario': '2025-03-24T08:00:00+00:00', 'id': 9, 'operador_id': {'empresa': {'id': 1, 'nome': 'TECSCI'}, 'id': 10, 'nome': 'Valciscley'}, 'rft': {'controladora_id': '0225020112193', 'erro_id': {'etapa': {'id': 2, 'nome': 'Teste de Comunicação'}, 'id': 3, 'nome': 'Falha no ADC'}, 'horario': '2025-03-24T08:00:00+00:00', 'id': 6, 'operador_id': {'empresa': {'id': 1, 'nom